## Proyecto de presentacion de Inteligencia Artificial
Mi tema : 





### Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as plt

: 

In [ ]:
titulos = pd.read_csv("csv/titles.csv")

: 